### About

This notebook uses the following frameworks
* Langchain

* Langgraph

### 1. Installing dependencies

In [ ]:
!pip install scikit-learn transformers

In [ ]:
!pip install -qU langchain_community langchain-qdrant langchain_huggingface

In [ ]:
!pip install rank_bm25

### 2. Local models

Loading the embedding model "sentence-camembert-large" from hugging face:

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.


In [2]:
embeddings.get_sentence_embedding_dimension()

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'get_sentence_embedding_dimension'

In [ ]:
### LLM
from langchain_ollama import ChatOllama

local_llm = "mistral:7b-instruct"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [ ]:
llm.invoke("hi")

### 3. Vector store

In [ ]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(url="http://localhost:6333")

In [ ]:
import json
import uuid
import hashlib
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# Helper function to convert string IDs to valid UUID-compatible IDs
def string_to_uuid(string_id):
    """Convert a string to a deterministic UUID by hashing it"""
    # Create MD5 hash of the string
    hash_object = hashlib.md5(string_id.encode())
    # Convert to hex
    hex_dig = hash_object.hexdigest()
    # Create a UUID from the hex string
    return uuid.UUID(hex_dig)

In [ ]:
# 4. Load your JSON file
try:
    with open('output_chunks.json', 'r', encoding='utf-8') as f:
        documents = json.load(f)
    print(f"✓ Loaded {len(documents)} documents from output_chunks.json")
except FileNotFoundError:
    print("Error: output_chunks.json file not found!")
    sys.exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in output_chunks.json!")
    sys.exit(1)

# 5. Create a collection
collection_name = "Auditron_legal_chunks"
print(f"Creating collection '{collection_name}'...")
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE),
)

# 6. Process the documents
print("Processing documents...")
total_chunks = sum(len(doc.get("chunks", [])) for doc in documents)
batch_size = 50
processed_count = 0
error_count = 0
batch_points = []
id_mapping = {}  # To store mapping between original IDs and UUID IDs

for doc_idx, doc in enumerate(documents):
    print(f"Processing document {doc_idx+1}/{len(documents)}")
    chunks = doc.get("chunks", [])
    
    for chunk_idx, chunk in enumerate(chunks):
        try:
            # Extract text and ID
            text = chunk.get("text", "")
            original_id = chunk.get("chunk_id", f"unknown_{doc_idx}_{chunk_idx}")
            
            if not text.strip():  # Skip empty chunks
                print(f"Warning: Empty text in chunk {original_id}")
                continue
            
            # Clean and truncate text if necessary to avoid index errors
            # Some models have maximum input length limitations
            max_text_length = 512  # Adjust based on your model's limitations
            text = text.strip()[:max_text_length]
            
            # Convert string ID to UUID
            point_id = string_to_uuid(original_id)
            
            # Save mapping
            id_mapping[str(point_id)] = original_id
            
            # Generate embedding with error handling
            try:
                embedding = model.encode(text, show_progress_bar=False)
            except Exception as embed_error:
                print(f"Embedding error for chunk {original_id}: {str(embed_error)}")
                # Try with a shorter text if it might be a length issue
                if len(text) > 200:
                    try:
                        shorter_text = text[:200]
                        print(f"Retrying with shorter text for {original_id}")
                        embedding = model.encode(shorter_text, show_progress_bar=False)
                    except Exception as retry_error:
                        print(f"Still failed with shorter text: {str(retry_error)}")
                        error_count += 1
                        continue
                else:
                    error_count += 1
                    continue
            
            # Create point with UUID
            point = PointStruct(
                id=str(point_id),
                vector=embedding.tolist(),
                payload={
                    "text": text,
                    "original_id": original_id,  # Keep original ID in payload
                    "structures": chunk.get("structures", []),
                    "document_path": chunk.get("document_path", []),
                    "metadata": chunk.get("metadata", {})
                }
            )
            
            # Add to batch
            batch_points.append(point)
            processed_count += 1
            
            # If batch is full, upload to Qdrant
            if len(batch_points) >= batch_size:
                qdrant.upsert(
                    collection_name=collection_name,
                    points=batch_points,
                )
                print(f"Uploaded batch: {processed_count}/{total_chunks} chunks ({error_count} errors so far)")
                batch_points = []
                
        except Exception as e:
            print(f"Error processing chunk {original_id}: {str(e)}")
            error_count += 1

# Upload any remaining points
if batch_points:
    qdrant.upsert(
        collection_name=collection_name,
        points=batch_points,
    )
    print(f"Uploaded final batch: {processed_count}/{total_chunks} chunks")

# Save ID mapping for reference (optional)
try:
    with open('id_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(id_mapping, f, indent=2)
    print("✓ Saved ID mapping to id_mapping.json")
except Exception as e:
    print(f"Warning: Could not save ID mapping: {str(e)}")

print(f"✅ Successfully processed {processed_count}/{total_chunks} chunks into Qdrant collection '{collection_name}'!")
print(f"Total errors encountered: {error_count}")

### 4. Retrieving

**4.1 Semantic search**

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import SearchParams
from langchain_core.embeddings import Embeddings

# 3. Create the vector store with LangChain
vector_store = QdrantVectorStore(
    client=qdrant,
    collection_name="Auditron_legal_chunks",  # Your collection name
    content_payload_key="text", 
    embedding=embeddings,
)
#    search_params=SearchParams(hnsw_ef=128)  # Your search params
# 4. Create the retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 5. Use the retriever
query = "Quel est le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020?"
documents = retriever.invoke(query)

# The retrieved documents will include content and metadata
for doc in documents:
    print(doc.page_content)  # Access the content
    print(doc.metadata)      # Access the metadata

### Document re-ranking

In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

### Retrieval Grader

# Doc grader instructions
doc_grader_instructions = """Vous êtes un modèle chargé d’évaluer la pertinence d’un document récupéré par rapport à une question utilisateur.

Si le document contient des mots-clés ou un sens sémantique lié à la question, considérez-le comme pertinent."""

# Grader prompt
doc_grader_prompt = """Voici le document récupéré : \n\n {document} \n\n Voici la question de l'utilisateur : \n\n {question}.

Évaluez soigneusement et objectivement si le document contient au moins une information pertinente en lien avec la question.

Retournez un JSON avec une clé unique, binary_score, qui sera 'oui' ou 'non' pour indiquer si le document contient au moins une information pertinente pour la question."""
# Test
question = "Quel est le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020?"

docs = retriever.invoke(question)
doc_txt = docs[0].page_content

doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

In [ ]:
doc_txt

In [ ]:
docs = retriever.invoke(question)
doc_txt = docs[2].page_content

doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

In [ ]:
doc_txt

**4.2 Implementig Hybrid Search**

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever  # Changed import location
from langchain_core.documents import Document

# Load all points from Qdrant
response = qdrant.scroll(
    collection_name="Auditron_legal_chunks",
    limit=10_000,  # Increase if needed, Qdrant supports pagination
    with_payload=True
)

# Convert Qdrant points to LangChain Documents
all_documents = []
for point in response[0]:  # response = (points, next_page_offset)
    payload = point.payload
    content = payload.get("text", "")
    metadata = {k: v for k, v in payload.items() if k != "text"}
    all_documents.append(Document(page_content=content, metadata=metadata))
    # Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(all_documents)
bm25_retriever.k = 3  # Retrieve top 5 results

# Create the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], 
    weights=[0.4, 0.6]  # You can adjust these weights based on performance
)


In [ ]:
# Example query
query = "Quel est le taux de retenue à la source sur les dividendes distribués par des sociétés tunisiennes à des personnes non résidentes, et comment une convention de non-double imposition peut-elle modifier ce taux pour les bénéficiaires d’une telle convention ?"
# Use the ensemble retriever
documents = ensemble_retriever.invoke(query)

# The retrieved documents will include content and metadata
for doc in documents:
    print(doc.page_content)  # Access the content
    print(doc.metadata)      # Access the metadata

In [ ]:
documents = ensemble_retriever.invoke(query)

In [ ]:
documents

**4.3 Relevancy threshold** 

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import SearchParams
from langchain_core.embeddings import Embeddings
from langchain_community.vectorstores import Qdrant
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.documents import Document

# Create the vector store with LangChain
vectorstore = QdrantVectorStore(
    client=qdrant,
    collection_name="Auditron_legal_chunks",  # Your collection name
    content_payload_key="text", 
    embedding=embeddings,
)
#    search_params=SearchParams(hnsw_ef=128)  # Your search params

# 4. Create the retriever from the vector store
qdrant_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.4
    }
)

# Load all points from Qdrant
response = qdrant.scroll(
    collection_name="Auditron_legal_chunks",
    limit=10_000,  # Increase if needed, Qdrant supports pagination
    with_payload=True
)

all_documents = []
for point in response[0]:  # response = (points, next_page_offset)
    payload = point.payload
    content = payload.get("text", "")
    metadata = {k: v for k, v in payload.items() if k != "text"}
    all_documents.append(Document(page_content=content, metadata=metadata))
    
# Set up your retrievers with the with_score parameter
bm25_retriever = BM25Retriever.from_documents(
    all_documents,
    k=5,
    with_score=True  # This returns scores with documents
)

# Combine them
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, qdrant_retriever], 
    weights=[0.4, 0.6]
)

In [ ]:
def retrieve_with_threshold(query):
    docs = ensemble_retriever.invoke(query)
    for doc in docs:
        print(f"Score: {doc.metadata.get('score', 'N/A')}, Content: {doc.page_content}")
    return docs


In [ ]:
# Usage
results = retrieve_with_threshold("Quel est le taux de la retenue à la source sur les dividendes versés à une personne morale résidente ?")

In [ ]:
results

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_core.embeddings import Embeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from typing import List, Any
from pydantic import Field
from langchain_huggingface import HuggingFaceEmbeddings

# 1. Updated Qdrant retriever with explicit embeddings
class QdrantScoreRetriever(BaseRetriever):
    vectorstore: QdrantVectorStore = Field(...)
    embeddings: Embeddings = Field(...)  # Explicit embeddings field
    k: int = Field(default=5)
    score_threshold: float = Field(default=0.4)

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Get embedding using the explicit embeddings model
        query_embedding = self.embeddings.embed_query(query)
        
        # Search Qdrant with score threshold
        results = self.vectorstore.client.search(
            collection_name=self.vectorstore.collection_name,
            query_vector=query_embedding,
            limit=self.k,
            score_threshold=self.score_threshold,
            with_payload=True
        )

        documents = []
        for result in results:
            content = result.payload.get("text", "")
            metadata = {k: v for k, v in result.payload.items() if k != "text"}
            metadata["score"] = result.score  # Store similarity score
            documents.append(Document(
                page_content=content,
                metadata=metadata
            ))
        
        return documents

# 2. Initialize components
qdrant = qdrant = QdrantClient(url="http://localhost:6333")
embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

# Create vector store
vectorstore = QdrantVectorStore(
    client=qdrant,
    collection_name="Auditron_legal_chunks",
    content_payload_key="text",
    embedding=embeddings,  # This might not be needed depending on your Qdrant setup
)

# 3. Create Qdrant retriever with explicit embeddings
qdrant_retriever = QdrantScoreRetriever(
    vectorstore=vectorstore,
    embeddings=embeddings,  # Pass embeddings explicitly
    k=5,
    score_threshold=0.4
)

# Load BM25 documents
response = qdrant.scroll(
    collection_name="Auditron_legal_chunks",
    limit=10_000,
    with_payload=True
)

all_documents = [
    Document(
        page_content=point.payload.get("text", ""),
        metadata={k: v for k, v in point.payload.items() if k != "text"}
    )
    for point in response[0]
]

bm25_retriever = BM25Retriever.from_documents(
    all_documents,
    k=5,
    with_score=True
)

# 4. Create ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, qdrant_retriever],
    weights=[0.4, 0.6]
)

# 5. Search function with threshold
def retrieve_with_threshold(query: str, threshold: float = 0.4) -> List[Document]:
    """Retrieve documents with combined score above threshold"""
    docs = ensemble_retriever.invoke(query)
    
    # Filter documents by combined score
    filtered_docs = [
        doc for doc in docs
        if doc.metadata.get("score", 0) >= threshold
    ]
    
    # Print results
    for i, doc in enumerate(filtered_docs, 1):
        print(f"Document {i} [Score: {doc.metadata['score']:.3f}]")
        print(doc.page_content[:300] + "...\n")
    
    return filtered_docs

# 6. Execute the search
results = retrieve_with_threshold(
    "Quel est le taux de la retenue à la source sur les dividendes versés à une personne morale résidente ?"
)

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.


Document 1 [Score: 0.658]
RETENUES A LA SOURCE ARTICLE 52 :

L'impôt sur le revenu et l'impôt sur les sociétés font l'objet d'une retenue à la source aux taux suivants :...

Document 2 [Score: 0.662]
b- Les retenues à la source L’impôt sur le revenu et l’impôt sur les sociétés font l’objet d’une retenue à la source, et ce, pour les montants qui sont dans le champ d’application de ladite retenue et payés par les personnes soumises à l’obligation d’effectuer la retenue à la source....

Document 3 [Score: 0.660]
Pour la détermination du prix (1) Ce taux s’applique aux revenus distribués à partir du 01/01/2018 et demeurent exonérés de l’impôt sur les bénéfices distribués les opérations de distribution de bénéfices à partir des fonds propres figurant au bilan de la société distributrice au 31 décembre 2013, à...

Document 4 [Score: 0.651]
L’impôt sur le revenu dû au titre de la plus-value Sous réserve des exonérations prévues par la législation en vigueur, la plus- value réalisée par les pe

C:\Users\foura\AppData\Local\Temp\ipykernel_23092\1034766933.py:23: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.vectorstore.client.search(


### 5. Generation

In [ ]:
### Generate

# Prompt
rag_prompt = """Vous êtes un assistant pour des tâches de question-réponse.

Voici le contexte à utiliser pour répondre à la question :

{context}

Réfléchissez soigneusement au contexte ci-dessus.

Maintenant, examinez la question de l'utilisateur :

{question}

Fournissez une réponse à cette question en utilisant uniquement le contexte ci-dessus.

Utilisez un maximum de trois phrases et gardez la réponse concise.

Réponse :"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
query = '''TEJ'''
docs = ensemble_retriever.invoke(query)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=query)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)


Hallucination grader:

In [ ]:
### Hallucination Grader

# Hallucination grader instructions
hallucination_grader_instructions = """

Vous êtes un enseignant en train de corriger un quiz.

Vous recevrez des FAITS et une RÉPONSE D'ÉLÈVE.

Voici les critères de notation à suivre :

(1) Assurez-vous que la RÉPONSE DE L'ÉLÈVE est bien fondée sur les FAITS.

(2) Assurez-vous que la RÉPONSE DE L'ÉLÈVE ne contient pas d'informations « hallucinées » qui sortent du cadre des FAITS.

Note :

Une note de oui signifie que la réponse de l'élève respecte tous les critères. C’est la note la plus élevée (meilleure).

Une note de non signifie que la réponse de l'élève ne respecte pas tous les critères. C’est la note la plus basse que vous pouvez attribuer.

Expliquez votre raisonnement étape par étape afin de garantir la justesse de votre raisonnement et de votre conclusion.

Évitez d’énoncer directement la bonne réponse dès le départ.
"""


# Grader prompt
hallucination_grader_prompt = """FAITS : \n\n {documents} \n\n RÉPONSE DE L'ÉLÈVE : {generation}.

Retournez un JSON avec deux clés :  
- binary_score : une valeur 'oui' ou 'non' indiquant si la RÉPONSE DE L'ÉLÈVE est bien fondée sur les FAITS.  
- explanation : une explication justifiant la note attribuée.
"""

# Test using documents and generation from above
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
    documents=docs_txt, generation=generation.content
)
result = llm_json_mode.invoke(
    [SystemMessage(content=hallucination_grader_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)
json.loads(result.content)

Answer grader

In [ ]:
### Answer Grader

# Answer grader instructions
answer_grader_instructions = """Vous êtes un enseignant en train de corriger un quiz.

Vous recevrez une QUESTION et une RÉPONSE D'ÉLÈVE.

Voici les critères de notation à suivre :

(1) La RÉPONSE DE L'ÉLÈVE aide à répondre à la QUESTION.

Note :

Une note de oui signifie que la réponse de l'élève respecte tous les critères. C’est la note la plus élevée (meilleure).

L’élève peut recevoir une note de oui même si la réponse contient des informations supplémentaires qui ne sont pas explicitement demandées dans la question.

Une note de non signifie que la réponse de l'élève ne respecte pas tous les critères. C’est la note la plus basse que vous pouvez attribuer.

Expliquez votre raisonnement étape par étape afin de garantir la justesse de votre raisonnement et de votre conclusion.

Évitez d’énoncer directement la bonne réponse dès le départ.
"""

# Grader prompt
answer_grader_prompt = """QUESTION : \n\n {question} \n\n RÉPONSE DE L'ÉLÈVE : {generation}.

Retournez un JSON avec deux clés :  
- binary_score : une valeur 'oui' ou 'non' indiquant si la RÉPONSE DE L'ÉLÈVE respecte les critères.  
- explanation : une explication justifiant la note attribuée.
"""


# Test
answer = generation.content

# Test using question and generation from above
answer_grader_prompt_formatted = answer_grader_prompt.format(
    question=question, generation=answer
)
result = llm_json_mode.invoke(
    [SystemMessage(content=answer_grader_instructions)]
    + [HumanMessage(content=answer_grader_prompt_formatted)]
)
json.loads(result.content)

### 6. ChatAgent with LangGraph